# MobileNet_v3 with Lite-RASSP



- This notebook contains a tensorflow.keras implementation of the MobileNet v3 with Segmentation head presented in the paper: https://arxiv.org/pdf/1905.02244.pdf


## 0 Imports

In [77]:
import tensorflow as tf
from tensorflow import keras
from keras import backend as K

## 1.0 Define model blocks

In [78]:
from keras.models import Model

from keras.layers import Conv2D, BatchNormalization, ReLU, DepthwiseConv2D, Activation, Input, Add, UpSampling2D
from keras.layers import GlobalAveragePooling2D, Reshape, Dense, Multiply, Softmax, AveragePooling2D,Softmax

from keras.utils.generic_utils import get_custom_objects


In [79]:
#define hard swish activation function
def hard_swish(x):
    return x * tf.nn.relu6(x + 3) / 6

In [80]:
get_custom_objects().update({'custom_activation': Activation(hard_swish)})

In [81]:
#squeeze and excitation block

def sq_n_ex(input_, r=4):

    '''
    param: input , ratio (r=4 in the paper)
    '''

    input_sNe_shape = (1, 1, input_._keras_shape[-1]) 
    sNe_layer = GlobalAveragePooling2D()(input_)
    sNe_layer = Reshape(input_sNe_shape)(sNe_layer)
    
    #ratio is used only in the first fully connected layer
    sNe_layer = Dense(filters // r, activation='relu', kernel_initializer='he_normal', use_bias=False)(sNe_layer)  
    #hard sigmoid in the second FC
    sNe_layer = Dense(filters, activation='hard_sigmoid', kernel_initializer='he_normal', use_bias=False)(sNe_layer)
    
    return Multiply()([input_, sNe_layer])


In [82]:
#bottleneck block


def bottleneck(input_,kernel, exp_size, out_dim,SE=False,NL='RE',s=1):
    
    '''
    params: 
    - input: network input
    - kernel: kernel size
    - exp_size: expansion size
    - out_dim: layer output dimension
    - SE: booelan. If the "squeeze and excite" block is present
    - NL: Non linear function used. RE= relu or HS=hard swish
    - s: stride
    '''
    
    #first layer
    
    x = Conv2D(exp_size, (1, 1), strides=(1, 1), padding='same')(input_)
    x = BatchNormalization()(x)
    if NL == 'RE':
        x = ReLU()(x)
    elif NL == 'HS':
        x = Activation(hard_swish)(x)
    
    
    #second layer
      
    x = DepthwiseConv2D(kernel_size=kernel, strides=(s,s), depth_multiplier=1, padding='same')(x)
    x = BatchNormalization()(x)
    
            
    if NL == 'RE':
        x = ReLU()(x)
    elif NL == 'HS':
        x = Activation(hard_swish)(x)
    
    #if squeeze and excite is in the block
    if SE:
        x = sq_n_ex(x)
        

    # third layer

    x = Conv2D(out_dim, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    
    #residual just if stride=1 and same dimension of x and f(x) 
    if s == 1 and K.int_shape(input_)[-1] == out_dim:
        x = Add()([x,input_])

    
    
    return x
    

## 2.0 Define model

In [83]:
def build_Model(in_dim, n_classes = 1000, include_top=True, model_version= 'large'):
    
    '''
    params:
    - in_dim: input dimension
    - n_class: number of classes for segmentation 
    - include_top: boolean. If true: include top
    - model_version: define model version of the mobilenet dfor features extraciton
    '''
    
    #input layer
    inputs = Input(shape=(in_dim))
    
    x = Conv2D(16,(3,3), padding='same', strides=2)(inputs)
    x = BatchNormalization()(x)
    
    if model_version=='large':
        x= bottleneck(x, (3,3), 16, 16 ,False,'RE',1)
        x= bottleneck(x, (3,3), 64, 24 ,False,'RE',2)
        x= bottleneck(x, (3,3), 72, 24 ,False,'RE',1)
        
        x= bottleneck(x, (5,5), 72, 40 ,True,'RE',2)
        x= bottleneck(x, (5,5), 120,120,True,'RE',1)
        x= bottleneck(x, (5,5), 120,120,True,'RE',1)
        
        x= bottleneck(x, (3,3), 240,80 ,False,'HS',2)
        x= bottleneck(x, (3,3), 200,80 ,False,'HS',1)
        x= bottleneck(x, (3,3), 184,80 ,False,'HS',1)
        
        x= bottleneck(x, (3,3), 184,80 ,False,'HS',1)
        x= bottleneck(x, (3,3), 480,112,True,'HS',1)
        x= bottleneck(x, (3,3), 672,112,True,'HS',1)
        
        x= bottleneck(x, (5,5), 672,160,True,'HS',2)
        x= bottleneck(x, (5,5), 960,160,True,'HS',1)
        x= bottleneck(x, (5,5), 960,160,True,'HS',1)
        
        if include_top:
            x = Conv2D(960,(1,1), padding='same', strides=1)(x)
            x = BatchNormalization()(x)
            x = Activation(hard_swish)(x)

            x = GlobalAveragePooling2D()(x)
            x = Reshape((1, 1, 960))(x)


            x = Conv2D(1280,(1,1), padding='same')(x)
            x = Activation(hard_swish)(x)
            x = Conv2D(n_classes, (1, 1), padding='same', activation='softmax')(x)
            
            
    elif model_version=='small':
        
        x= bottleneck(x, (3,3), 16,16 ,True,'RE',2)
        x= bottleneck(x, (3,3), 72,24 ,False,'RE',2)
        x= bottleneck(x, (3,3), 88,24 ,False,'RE',1)

        x= bottleneck(x, (5,5), 96, 40 ,True,'HS',2)
        x= bottleneck(x, (5,5), 240,40 ,True,'HS',1)
        x= bottleneck(x, (5,5), 240,40 ,True,'HS',1)

        x= bottleneck(x, (5,5), 120,48 ,True,'HS',1)
        x= bottleneck(x, (5,5), 144,48 ,True,'HS',1)
        x= bottleneck(x, (5,5), 288,96 ,True,'HS',2)

        x= bottleneck(x, (5,5), 576,96 ,True,'HS',1)
        x= bottleneck(x, (5,5), 576,96 ,True,'HS',1)

        if include_top:
            x = Conv2D(576,(1,1), padding='same', strides=1)(x)
            x = BatchNormalization()(x)
            x = Activation(hard_swish)(x)

            x = GlobalAveragePooling2D()(x)
            x = Reshape((1, 1, 576))(x)


            x = Conv2D(1024,(1,1), padding='same')(x)
            x = Activation(hard_swish)(x)
            x = Conv2D(n_classes, (1, 1), padding='same', activation='softmax')(x)
        

 
    else:
        raise TypeError("Wrong model version inserted. Please insert large or small.") 
        
    
    #finally define the model
    model = Model(inputs=inputs, outputs=x)
    
    return model

In [73]:
K.clear_session()

#example
model = build_Model((224,224,3),model_version='large')   #X_train[0].shape

In [74]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 16) 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 112, 112, 16) 272         batch_normalization_1[0][0]      
____________________________________________________________________________________________

## Adding Segmentation head

In [70]:

def add_seg_head(base_model, n_class=19,  model_version= 'large'): 
    '''
    params:
    - base_model: mobileNetv3 already built
    - n_class: number of classes for segmentation 
    - model_version: define model version of the mobilenet dfor features extraciton
    '''
    if model_version=='large':
        layer_name_1_8='activation_1'
        layer_name_1_16='activation_13'
    
    elif model_version=='small':
        layer_name_1_8='activation_1'
        layer_name_1_16='activation_11'
    
    else:
        raise TypeError("Wrong model version inserted. Please insert large or small.") 
        
        
        
    #1/8 resolution output
    
    out_1_8= base_model.get_layer(layer_name_1_8).output
    
    #1/16 resolution output
    
    out_1_16= base_model.get_layer(layer_name_1_16).output
    
    
    # branch1
    x1 = Conv2D(128, (1, 1))(out_1_16)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)  
    
    # branch2
    #!!! important: the pool size and the strides of the next layer must be resized in case of input dimension
    #               equal to the one in the paper (224,224,3)
    
    x2 = AveragePooling2D(pool_size=(49, 49), strides=(16, 20),data_format='channels_last')(out_1_16)
    x2 = Conv2D(128, (1, 1))(x2)
    x2 = Activation('sigmoid')(x2)
    x2 = UpSampling2D(size=(int(x1.shape[1]), int(x1.shape[2])),data_format='channels_last',interpolation="bilinear")(x2)
       
    # branch3
    x3 = Conv2D(n_class, (1, 1))(out_1_8)
    
    # multiply
    m1 = Multiply()([x1, x2])
    m1 = UpSampling2D(size=(2, 2),data_format='channels_last',interpolation="bilinear")(m1)
    m1 = Conv2D(n_class, (1, 1))(m1)

    # add
    m2 = Add()([m1, x3])
    
    #adding UPsampling
    #!!! important: this upsampling is not part of the network presented in the paper
    #              but it's necessary for having output shape = input shape
    m2 = UpSampling2D(size=(8, 8),data_format='channels_last',interpolation="bilinear")(m2)

    # predictions 
    predictions =Softmax()(m2)  

    # final model
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model



In [75]:
#model with segmentation head
model_with_SH=add_seg_head(model,model_version= 'large')

In [76]:
model_with_SH.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 16) 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 112, 112, 16) 272         batch_normalization_1[0][0]      
____________________________________________________________________________________________